In [5]:
import pandas as pd
import numpy as np
import requests
import json
from selenium import webdriver
from bs4 import BeautifulSoup

In [6]:
def get_elements(url):
    driver = webdriver.Chrome("chromedriver.exe")
    desciption=''
    deadline=''
    domain=''
    level=''
    contract=''
    country=''
    driver.get(url)
    content = driver.page_source
    soup = BeautifulSoup(content)
    i=0
    for a in soup.findAll('div',attrs={'class':'root responsivegrid'}):
        for b in a.findAll('div',attrs={'class':'parsys aem-GridColumn aem-GridColumn--default--12'}):
            for c in b.findAll('div',attrs={'class':'l-grid__column-medium-9 l-grid__column-large-6 js-carousel--product-tile u-margin-top-sm'}):
                for d in c.findAll('div',attrs={'class':'l-grid__row c-tile__product-tile--wrapper u-margin-top-lg'}):
                    for e in d.findAll('div',attrs={'class':'l-grid__column-medium-4 c-tile__product-tile--slide'}):
                        for f in e.findAll('div',attrs={'class':'c-tile--product-teaser'}):
                            for g in f.findAll('div',attrs={'class':'c-copy c-tile__product--title u-padding-top-md'}):
                                if i==1:
                                    deadline=g.text
                                elif i==2:
                                    country=g.text
                                elif i==3:
                                    contract=g.text
                                elif i==4:
                                    level=g.text
                                elif i==5:
                                    domain=g.text
                                i+=1
    description=''
    for a in soup.findAll('div',attrs={'class':'c-tabs__content-item-content c-convert_content'}):
        description+=a.text
    return deadline,country,contract,level,domain,description

In [7]:
def get_docs(keyword):
    header_list = ['Country','Description','Title','Company','Deadline','Contract','Level','Domain']
    r = requests.get('https://careers.allianz.com/fr_FR/search/_jcr_content.fulltextsearch.json?q='+keyword+'&start=0&fq=az_aemTags&fbclid=IwAR3OzeJ-9bSPEc-CKl3J24otUtxlAcd5zoOhYdkNeJHjbMBxcUyffUZHdUg')
    arr=json.loads(r.text)
    docs=pd.DataFrame(arr['response']['docs'])
    print(len(docs))
    n=int(arr['response']['numFound'])
    if n==0:
        return pd.DataFrame(columns=header_list)
    docs.drop(columns=['az_getAQuoteUrl','az_type','az_uri','id'],inplace=True)
    docs.rename(columns={'az_breadcrumb':'Country','az_description':'Description','az_title':'Title'},inplace=True)
    infos=list(docs['Country'])
    docs['Company']=[l[2] for l in infos]
    url='https://careers.allianz.com/'
    docs['Description']=['' if l=='pageDescMsg' else l for l in docs['Description'] ]
    for aux in ['Deadline','Contract','Level','Domain']:
        docs[aux]=''
    for i in range(0,len(infos)):
        deadline,country,contract,level,domain,description=get_elements(url+infos[i][5])
        docs['Deadline'][i]=deadline
        docs['Country'][i]=country
        docs['Contract'][i]=contract
        docs['Domain'][i]=domain
        docs['Description'][i]+=description
        docs['Level'][i]=level
    return docs

In [ ]:
docs=get_docs('juridique')
docs2=get_docs('fiscalite')

15


In [49]:
docs.head()

,Country,Description,Title,Company,Deadline,Contract,Level,Domain
0,"\nParis,Paris, France\n",Postulez maintenant en tant que Responsable ju...,Responsable juridique département assurance (H...,Allianz-France,\nValable jusqu'au31.12.2019\n,\n \n notAvailable\n \n...,\n \n Manager\n \n,\n \n Juridique\n \n
1,"\nEtoy,Vaud, Suisse\n","\n\n En tant qu’assistant/e juridique, ...",Assistant/e Juridique 80-100% pour la CAP Prot...,CAP-Rechtsschutz-AG,\nValable jusqu'aunotAvailable\n,\n \n CDI / Temps Plein\n ...,\n \n Expérimenté\n \n ...,\n \n Administration\n ...
2,"\nEtoy,Vaud, Suisse\n","\n\n En tant que juriste, vous êtes res...",Juriste de 80 à 100% pour la CAP Protection Ju...,CAP-Rechtsschutz-AG,\nValable jusqu'aunotAvailable\n,\n \n CDI / Temps Plein\n ...,\n \n Jeune diplômé\n \...,\n \n Juridique\n \n
3,"\nParis,Paris, France\n","\n\n Protexia France, créée en 1991, es...",ALTERNANCE Juriste H/F-Paris-Allianz France | ...,Allianz-France,\nValable jusqu'au31.10.2019\n,\n \n notAvailable\n \n...,\n \n Alternant\n \n,\n \n Juridique\n \n
4,"\nParis,Paris, France\n",Postulez maintenant en tant que Juriste Corpor...,Juriste Corporate H/F-Paris-Allianz France | A...,Allianz-France,\nValable jusqu'au17.11.2019\n,\n \n CDI / Temps Plein\n ...,\n \n Expérimenté\n \n ...,\n \n Juridique\n \n


In [52]:
docs.to_csv('docs.csv',index=False)